In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel, BertConfig

C:\Users\USER\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Init

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
configuration = BertConfig()
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True).to(device)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', use_fast=True)
print("Device: ",device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Device:  cuda


## Read Pickle

In [29]:
"""
Required dataframe format:
[column name]       [dtype]
AppID               int
UserID              int
Like                int
Review              String
"""
data = pd.read_pickle(r'../data/filtered_reviews_group.pkl')
# Change dtype of columns in df
# data.reset_index(drop=True, inplace=True)
# data.drop(["RefValue", "VoteUp"], axis=1, inplace=True)
# data["UserID"] = data["UserID"].astype("int64")
# data["AppID"] = data["UserID"].astype("int64")

"""
TODO dataframe format:
[column name]       [dtype]
SplitReview         list
SplitReview_emb     np.array
LDA_group           list
"""
# TODO Columns
# data["SplitReview"] = ""
# data["SplitReview_emb"]=""
# data["LDA_group"]=""

print(data.dtypes)
data

AppID               int64
UserID              int64
Like                int64
Review             object
SplitReview        object
LDA_group          object
SplitReview_emb    object
dtype: object


,AppID,UserID,Like,Review,SplitReview,LDA_group,SplitReview_emb
0,730,76561198058115931,0,Trust Factor makes this game unplayable.I'm al...,"[Trust Factor makes this game unplayable, I'm ...","[4, 3, 3, 3, 0, 0, 0, 0, 0, 0]",
1,730,76561198046999701,1,Counter Strike on Steam Deck as our lord Gaben...,[Counter Strike on Steam Deck as our lord Gabe...,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0]",
2,730,76561198004492389,1,Has come a long way from where it was with ram...,[Has come a long way from where it was with ra...,"[1, 4, 4, 4, 3, 2, 4, 1, 0, 0]",
3,730,76561198130196263,0,i hate the community of this game with a burni...,[i hate the community of this game with a burn...,"[4, 0, 0, 0, 0, 0, 0, 0, 0, 0]",
4,730,76561198066129673,1,You know what's even more amazing?Ingredients\...,"[You know what's even more amazing, 150g salte...","[3, 4, 4, 5, 4, 1, 3, 4, 5, 4]",
...,...,...,...,...,...,...,...
117710,1972440,76561197970447700,1,This game is pretty damn good.I like the aesth...,"[This game is pretty damn good, I like the aes...","[2, 2, 3, 3, 5, 4, 2, 3, 2, 0]",
117711,1972440,76561198027873712,1,Current price of about 5 bucks is really fair....,[Current price of about 5 bucks is really fair...,"[1, 3, 4, 1, 1, 5, 5, 5, 5, 5]",
117712,1972440,76561198151395732,1,first thoughts after about an hour are that th...,[first thoughts after about an hour are that t...,"[4, 4, 5, 2, 3, 2, 3, 4, 2, 2]",
117713,1972440,76561197996690770,1,"Needs quick restart added, and change shields ...","[Needs quick restart added, and change shields...","[5, 0, 0, 0, 0, 0, 0, 0, 0, 0]",


In [36]:
data["SplitReview"][2]

['Has come a long way from where it was with rampant cheating, griefing and smurfing',
 'I spent the better half of two years recording and playing nearly every match (CSGO Competitive Extreme)',
 'I even made a good number of friends from just playing Competitive -- no joke',
 "That being said, an anti-cheat alone can't keep people from sharing accounts, hopping on brand new ones when they get banned, or smurf too well",
 "Some people straight up throw the match because they're pissed off, or drunk, or high, but it doesn't happen too often for me to say it more than in passing",
 "I'm not a smoker, but this game is one good toke off the weed pen",
 "It'll get you buzzed, but if you rely on it too much will fuck you up permanently",
 "But that's why they call it Global Offensive"]

In [167]:
# 確定 threshold 後檢查有多少個 App、User
app_reviews = data['AppID'].value_counts()
user_reviews = data['UserID'].value_counts()
APP_THRESHOLD = 200
USER_THERSHOLD = 20
filter_apps = app_reviews[app_reviews > APP_THRESHOLD].index[:]
temp = data[ data['AppID'].isin(filter_apps) ]
filter_users = user_reviews[user_reviews > USER_THERSHOLD].index[:]
filter_data = temp[ temp['UserID'].isin(filter_users) ]
print('Apps: ', len(filter_data['AppID'].unique()), 'Users: ', len(filter_data['UserID'].unique()))
print('Total reviews: ',len(filter_data['AppID']))

Apps:  84 Users:  2575
Total reviews:  20284


In [168]:
# 查看各遊戲所擁有總評論數區間
unique, counts = np.unique(data["AppID"], return_counts=True)
count_dict = dict(zip(unique, counts))
for i in range(0, 1000, 100):
    print(f"{i}-{i+99}:\t", len(list(key for key, value in count_dict.items() if i <= value and value < i+99)))
print(f">1000:\t\t", len(list(key for key, value in count_dict.items() if 1000 < value)))

0-99:	 1313
100-199:	 248
200-299:	 63
300-399:	 14
400-499:	 8
500-599:	 0
600-699:	 1
700-799:	 0
800-899:	 0
900-999:	 0
>1000:		 0


# LDA Grouping

In [230]:
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from sklearn.feature_extraction.text import TfidfVectorizer


def stemmer_with_delete_stopword(split_sentences):
    vectorizer = TfidfVectorizer(stop_words = "english")
    stop_list = list(vectorizer.get_stop_words())
    porter_stemmer = PorterStemmer()
    all_stem_sents=[]
    for review in split_sentences:
        review_stem_list = []
        for sent in review:
            sent_stem_list =[]
            for word in sent.split(" "):
                if len(word)>2:
                    if word not in stop_list:
                        sent_stem_list.append(porter_stemmer.stem(word))
            review_stem_list.append(sent_stem_list)
        all_stem_sents.append(review_stem_list)
        
    return all_stem_sents


def LDAGrouping(reviews):
    all_sents = []
    for review in reviews:
        for sentence in review:
            all_sents.append(sentence)
    dictionary = corpora.Dictionary(all_sents)
    corpus = [dictionary.doc2bow(sent) for sent in all_sents]
    lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5)
    group_results = []
    for sents in reviews:
        single_corpus = [dictionary.doc2bow(sent) for sent in sents]
        sents_group_result = []
        for scores in lda.inference(single_corpus)[0]:
            # scores.argmax()+1 --> Retain group:0 for no meaning sentences
            sents_group_result.append(scores.argmax()+1)
        group_results.append(sents_group_result)

    return group_results

def pad_and_trunc(group_results, *, max_sentence):
    #max number of sentences in a review
    MAX_SENTENCE = max_sentence
    result_list = []
    for i, result in enumerate(group_results):
        if len(result) >= MAX_SENTENCE:
            result = result[:10]
        else:
            result.extend([0]*(MAX_SENTENCE-len(result)))
        result_list.append(result)
    result_list = np.array(result_list)
    return result_list

In [227]:
clean_reviews = stemmer_with_delete_stopword(list(data["SplitReview"]))
group_list = LDAGrouping(clean_reviews) # Training might take a little bit time 
data

完成第0筆
完成第1筆
完成第2筆
完成第3筆
完成第4筆
完成第5筆
完成第6筆
完成第7筆
完成第8筆
完成第9筆
完成第10筆
完成第11筆
完成第12筆
完成第13筆
完成第14筆
完成第15筆
完成第16筆
完成第17筆
完成第18筆
完成第19筆
完成第20筆
完成第21筆
完成第22筆
完成第23筆
完成第24筆
完成第25筆
完成第26筆
完成第27筆
完成第28筆
完成第29筆
完成第30筆
完成第31筆
完成第32筆
完成第33筆
完成第34筆
完成第35筆
完成第36筆
完成第37筆
完成第38筆
完成第39筆
完成第40筆
完成第41筆
完成第42筆
完成第43筆
完成第44筆
完成第45筆
完成第46筆
完成第47筆
完成第48筆
完成第49筆
完成第50筆
完成第51筆
完成第52筆
完成第53筆
完成第54筆
完成第55筆
完成第56筆
完成第57筆
完成第58筆
完成第59筆
完成第60筆
完成第61筆
完成第62筆
完成第63筆
完成第64筆
完成第65筆
完成第66筆
完成第67筆
完成第68筆
完成第69筆
完成第70筆
完成第71筆
完成第72筆
完成第73筆
完成第74筆
完成第75筆
完成第76筆
完成第77筆
完成第78筆
完成第79筆
完成第80筆
完成第81筆
完成第82筆
完成第83筆
完成第84筆
完成第85筆
完成第86筆
完成第87筆
完成第88筆
完成第89筆
完成第90筆
完成第91筆
完成第92筆
完成第93筆
完成第94筆
完成第95筆
完成第96筆
完成第97筆
完成第98筆
完成第99筆
完成第100筆
完成第101筆
完成第102筆
完成第103筆
完成第104筆
完成第105筆
完成第106筆
完成第107筆
完成第108筆
完成第109筆
完成第110筆
完成第111筆
完成第112筆
完成第113筆
完成第114筆
完成第115筆
完成第116筆
完成第117筆
完成第118筆
完成第119筆
完成第120筆
完成第121筆
完成第122筆
完成第123筆
完成第124筆
完成第125筆
完成第126筆
完成第127筆
完成第128筆
完成第129筆
完成第130筆
完成第131筆
完成第132筆
完成第133筆
完成第134筆
完成第135筆
完成第136筆
完成第137筆
完成第138

,AppID,UserID,Like,RefValue,VoteUp,Review,SplitReview,SplitReview_emb,LDA_group
0,76561198058115931,76561198058115931,0,0.554770,3,Trust Factor makes this game unplayable.I'm al...,"[Trust Factor makes this game unplayable, I'm ...",,"[4, 3, 3, 3, 0, 0, 0, 0, 0, 0]"
1,76561198046999701,76561198046999701,1,0.000000,0,Counter Strike on Steam Deck as our lord Gaben...,[Counter Strike on Steam Deck as our lord Gabe...,,"[3, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,76561198004492389,76561198004492389,1,0.531378,3,Has come a long way from where it was with ram...,[Has come a long way from where it was with ra...,,"[1, 4, 4, 4, 3, 2, 4, 1, 0, 0]"
3,76561198130196263,76561198130196263,0,0.000000,0,i hate the community of this game with a burni...,[i hate the community of this game with a burn...,,"[4, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,76561198066129673,76561198066129673,1,0.000000,0,You know what's even more amazing?Ingredients\...,"[You know what's even more amazing, 150g salte...",,"[3, 4, 4, 5, 4, 1, 3, 4, 5, 4]"
...,...,...,...,...,...,...,...,...,...
117710,76561197970447700,76561197970447700,1,0.000000,0,This game is pretty damn good.I like the aesth...,"[This game is pretty damn good, I like the aes...",,"[2, 2, 3, 3, 5, 4, 2, 3, 2, 0]"
117711,76561198027873712,76561198027873712,1,0.000000,0,Current price of about 5 bucks is really fair....,[Current price of about 5 bucks is really fair...,,"[1, 3, 4, 1, 1, 5, 5, 5, 5, 5]"
117712,76561198151395732,76561198151395732,1,0.000000,0,first thoughts after about an hour are that th...,[first thoughts after about an hour are that t...,,"[4, 4, 5, 2, 3, 2, 3, 4, 2, 2]"
117713,76561197996690770,76561197996690770,1,0.000000,0,"Needs quick restart added, and change shields ...","[Needs quick restart added, and change shields...",,"[5, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [235]:
# Save the LDA grouping result
data.to_pickle(r"../data/filtered_reviews_group.pkl")

# Bert Encode

### 1. Split every review to sentences.  

In [9]:
import re

def review_to_sentences(review):
    """
    split review into sentences contained by a list
    param: review (String)
    output: sentences (list of word)
    """
    sentences = review.splitlines()
    sentences = list(filter(None, sentences))
    tmp = []
    for sent in sentences:
        sent = re.split(r' *[\.\?!][\'"\)\]]* *', sent)
        tmp.extend(sent)
    # delete sentence less than 10 words 
    sentences = list(filter(lambda x:len(x.split())>=10, tmp))
    return sentences

In [ ]:
list_split_sentences =  [review_to_sentences(review) for review in data["Review"]]
data["SplitReview"] = list_split_sentences

### 2. Init Bert and encode methods

In [15]:
import re
device = "cuda" if torch.cuda.is_available() else "cpu"
args = {
        "device" : device,
        "data_dir" : r'../data/filtered_reviews_with_split.pkl',
        "data_chunks_dir" : r'../data/chunks',
        "emb_dim" : 768,
        "max_word" : 25,
        "max_sentence" : 10,
        "max_review_user" : 10,
        "max_review_item" : 30,
        "epoch" : 5,
        "batch_size": 32,
        "bert_configuration" : BertConfig(),
        "bert_model" : BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True).to(device),
        "bert_tokenizer" : BertTokenizer.from_pretrained('bert-base-uncased', use_fast=True),
    }

def padding_to_tagert_dimension(input_tensor, sent_len, word_len, word_dim):
    """
    Set input_tensor to specified dim with zero padding, and flatten it
    ex: [3, 25, 768] -> [10, 25, 768] -> [250, 768]
    """
    target_emb = torch.zeros(sent_len, word_len, word_dim)
    target_emb[:input_tensor.size(dim=0), :, :] = input_tensor
    target_emb = torch.flatten(target_emb, start_dim=0, end_dim=1)
    
    return target_emb

def bert_encode(review_split, args):
    """
    Encode splitted review to bert embedding
    return embedding of review padded with zero
    """
    emb_list = []
    for i, sentence in enumerate(review_split):
        if i == args["max_sentence"]: break
        sentence_encode = args["bert_tokenizer"](
            sentence,
            return_attention_mask = True,
            max_length = args["max_word"],
            truncation = True,
            padding = "max_length",
            return_tensors = 'pt'
            )
        for k,v in sentence_encode.items():
            sentence_encode[k] = v.to(args["device"])
        with torch.no_grad():
            outputs = args["bert_model"](**sentence_encode)
        sentence_emb = outputs[2][-1].to(args["device"])
        emb_list.append(sentence_emb)
    review_emb = torch.cat(emb_list, 0)
    pad_review_emb = padding_to_tagert_dimension(review_emb, args["max_sentence"], args["max_word"], args["emb_dim"])
    return pad_review_emb

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 3. Encode splited sentences and save into multiple chunks of H5DF

In [30]:
reviews = data["SplitReview"]
tmp = []
for i, review in enumerate(reviews):
    review_emb = bert_encode(review, args)
    review_emb = np.asarray(review_emb)
    data.at[i, "SplitReview_emb"] = review_emb
    print("執行至第",i ,"筆")
    if (i+1)%1000 == 0:
        start = i-999
        end = i+1
        print("存檔中...")
        data[start:end].to_hdf(f'../data/chunks/filtered_reviews_{start}-{end-1}.h5', key="df", mode="w")
    elif (i+1)==len(reviews):
        start = 117000
        end = i+1
        print("存檔中...")
        data[start:end].to_hdf(f'../data/chunks/filtered_reviews_{start}-{end-1}.h5', key="df", mode="w")

執行至第 0 筆
執行至第 1 筆
執行至第 2 筆
執行至第 3 筆
執行至第 4 筆
執行至第 5 筆
執行至第 6 筆
執行至第 7 筆
執行至第 8 筆
執行至第 9 筆
執行至第 10 筆
執行至第 11 筆
執行至第 12 筆
執行至第 13 筆
執行至第 14 筆
執行至第 15 筆
執行至第 16 筆
執行至第 17 筆
執行至第 18 筆
執行至第 19 筆
執行至第 20 筆
執行至第 21 筆
執行至第 22 筆
執行至第 23 筆
執行至第 24 筆
執行至第 25 筆
執行至第 26 筆
執行至第 27 筆
執行至第 28 筆
執行至第 29 筆
執行至第 30 筆
執行至第 31 筆
執行至第 32 筆
執行至第 33 筆
執行至第 34 筆
執行至第 35 筆
執行至第 36 筆
執行至第 37 筆
執行至第 38 筆
執行至第 39 筆
執行至第 40 筆
執行至第 41 筆
執行至第 42 筆
執行至第 43 筆
執行至第 44 筆
執行至第 45 筆
執行至第 46 筆
執行至第 47 筆
執行至第 48 筆
執行至第 49 筆
執行至第 50 筆
執行至第 51 筆
執行至第 52 筆
執行至第 53 筆
執行至第 54 筆
執行至第 55 筆
執行至第 56 筆
執行至第 57 筆
執行至第 58 筆
執行至第 59 筆
執行至第 60 筆
執行至第 61 筆
執行至第 62 筆
執行至第 63 筆
執行至第 64 筆
執行至第 65 筆
執行至第 66 筆
執行至第 67 筆
執行至第 68 筆
執行至第 69 筆
執行至第 70 筆
執行至第 71 筆
執行至第 72 筆
執行至第 73 筆
執行至第 74 筆
執行至第 75 筆
執行至第 76 筆
執行至第 77 筆
執行至第 78 筆
執行至第 79 筆
執行至第 80 筆
執行至第 81 筆
執行至第 82 筆
執行至第 83 筆
執行至第 84 筆
執行至第 85 筆
執行至第 86 筆
執行至第 87 筆
執行至第 88 筆
執行至第 89 筆
執行至第 90 筆
執行至第 91 筆
執行至第 92 筆
執行至第 93 筆
執行至第 94 筆
執行至第 95 筆
執行至第 96 筆
執行至第 97 筆
執行至第 98 筆
執行至第 99 筆
執行至第 100 筆

C:\Users\USER\AppData\Local\Temp\ipykernel_6568\2553433121.py:12: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['Review', 'SplitReview', 'LDA_group', 'SplitReview_emb'], dtype='object')]

  data[start:end].to_hdf(f'../data/chunks/filtered_reviews_{start}-{end-1}.h5', key="df", mode="w")


執行至第 1000 筆
執行至第 1001 筆
執行至第 1002 筆
執行至第 1003 筆
執行至第 1004 筆
執行至第 1005 筆
執行至第 1006 筆
執行至第 1007 筆
執行至第 1008 筆
執行至第 1009 筆
執行至第 1010 筆
執行至第 1011 筆
執行至第 1012 筆
執行至第 1013 筆
執行至第 1014 筆
執行至第 1015 筆
執行至第 1016 筆
執行至第 1017 筆
執行至第 1018 筆
執行至第 1019 筆
執行至第 1020 筆
執行至第 1021 筆
執行至第 1022 筆
執行至第 1023 筆
執行至第 1024 筆
執行至第 1025 筆
執行至第 1026 筆
執行至第 1027 筆
執行至第 1028 筆
執行至第 1029 筆
執行至第 1030 筆
執行至第 1031 筆
執行至第 1032 筆
執行至第 1033 筆
執行至第 1034 筆
執行至第 1035 筆
執行至第 1036 筆
執行至第 1037 筆
執行至第 1038 筆
執行至第 1039 筆
執行至第 1040 筆
執行至第 1041 筆
執行至第 1042 筆
執行至第 1043 筆
執行至第 1044 筆
執行至第 1045 筆
執行至第 1046 筆
執行至第 1047 筆
執行至第 1048 筆
執行至第 1049 筆
執行至第 1050 筆
執行至第 1051 筆
執行至第 1052 筆
執行至第 1053 筆
執行至第 1054 筆
執行至第 1055 筆
執行至第 1056 筆
執行至第 1057 筆
執行至第 1058 筆
執行至第 1059 筆
執行至第 1060 筆
執行至第 1061 筆
執行至第 1062 筆
執行至第 1063 筆
執行至第 1064 筆
執行至第 1065 筆
執行至第 1066 筆
執行至第 1067 筆
執行至第 1068 筆
執行至第 1069 筆
執行至第 1070 筆
執行至第 1071 筆
執行至第 1072 筆
執行至第 1073 筆
執行至第 1074 筆
執行至第 1075 筆
執行至第 1076 筆
執行至第 1077 筆
執行至第 1078 筆
執行至第 1079 筆
執行至第 1080 筆
執行至第 1081 筆
執行至第 1082 筆
執行至第

KeyboardInterrupt: 

### Show Bert Encode Result

In [130]:
tmp = pd.read_hdf(r"../data/chunks/filtered_reviews_0-999.h5")
print("Emb shape: ", tmp["SplitReview_emb"][0].shape)
tmp["SplitReview_emb"]

(250, 768)

In [117]:
a = np.array(tmp["LDA_group"][0])
np.where(a==3)

(array([1, 2, 3], dtype=int64),)